In [239]:
import h5py 
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import imageio
from matplotlib.gridspec import GridSpec
import os
import ffmpy

In [32]:
jet_pdb = pd.read_csv('/home/kitadam/ENR_Sven/moxie/data/raw/pedestal-database.csv')
important_info = jet_pdb[['shot', 't1', 't2', 'neseparatrixfromexpdata10^19(m^-3)', 'error_neseparatrixfromexpdata10^19(m^-3)','neseparatrixfromfit10^19(m^-3)',
 'error_neseparatrixfromfit10^19(m^-3)','FLAG:HRTSdatavalidated']]
final_pulse_list = important_info[(important_info['neseparatrixfromexpdata10^19(m^-3)'] != 0.0) & (important_info['neseparatrixfromexpdata10^19(m^-3)'] != -1.0) &  (important_info['shot'] >= 79000) &  (important_info['FLAG:HRTSdatavalidated'] > 0)]


In [240]:
# final_pulse_list[final_pulse_list['shot'] == 83357]
print(final_pulse_list[['shot', 't1', 't2']])

         shot         t1         t2
285   79192.0  54.498299  55.502201
286   79192.0  56.032001  58.535702
287   79193.0  54.506699  56.505100
288   79193.0  57.252998  58.790600
289   79195.0  54.555401  55.994400
...       ...        ...        ...
3552  98004.0  49.951801  51.694801
3553  98004.0  48.954700  49.924999
3554  98004.0  52.463902  52.935600
3555  98005.0  48.910599  49.771599
3556  98005.0  50.351898  51.437599

[2191 rows x 3 columns]


In [237]:
def get_window_mparams(mp_values, mp_times, t1, t2):
    windowed_idx = np.logical_and(mp_times > t1, mp_times < t2)
    new_values = mp_values[windowed_idx]
    new_times = mp_times[windowed_idx]
    return new_values, new_times

def get_window_profile_times(profiles, profiles_times, t1, t2):
    time_idx = np.logical_and(profiles_times > t1, profiles_times < t2)
    return profiles_times[time_idx], profiles[time_idx]

def average_machine_with_times(wind_times, mp_values, mp_times): 
    delta_T = 0.05002594
    sampled_vals = []
    for time in wind_times: 
        aggregation_idx = np.logical_and(mp_times < time, mp_times > time - delta_T)
        aggregation_vals = mp_values[aggregation_idx]
        
        sampled_vals.append(np.mean(aggregation_vals))
    return np.array(sampled_vals)
    # pass 

def plot_mp_pulses():
    pass 
    
def create_prof_mp_gif(window_profiles, prof_radius, window_times, raw_mp1_val, raw_mp1_time, raw_mp2_val, raw_mp2_time, window_mp1, window_mp2, t1, t2):
    filenames = []
    delta_T = 0.05002594
    mp1_min, mp1_max = min(window_mp1) - 2*np.std(window_mp1), max(window_mp1) + 2*np.std(window_mp1)
    mp2_min, mp2_max = min(window_mp2) - 2*np.std(window_mp2), max(window_mp2) + 2*np.std(window_mp2)
    prof_max = max(map(max, window_profiles))
    
    
    for t in range(len(window_times) - 1):
        fig = plt.figure(figsize=(12, 8), dpi=400)
        gs = GridSpec(2, 2, figure=fig)
        prof_ax = fig.add_subplot(gs[0,:])
        mp1_ax = fig.add_subplot(gs[1, 0])
        mp2_ax = fig.add_subplot(gs[1, 1])
        
        
        prof_ax.scatter(prof_radius, windowed_profiles[t])
        
        # print(raw_mp1_val, raw_mp1_time)
        
        mp1_ax.scatter(raw_mp1_time,raw_mp1_val)
        mp2_ax.scatter(raw_mp2_time, raw_mp2_val)
        
        mp1_ax.axvspan(window_times[t] - delta_T, window_times[t], alpha=0.5, color='grey')
        mp2_ax.axvspan(window_times[t] - delta_T, window_times[t], alpha=0.5, color='grey')
        
        mp1_ax.scatter(window_times[t], window_mp1[t], color='r')
        mp2_ax.scatter(window_times[t], window_mp2[t], color='r')
        
        prof_ax.set_ylim(-1e19, 1.25e20)
        mp1_ax.set_xlim(t1 - 0.2, t2 +0.2)
        mp2_ax.set_xlim(t1 - 0.2, t2 +0.2)
        mp1_ax.set_ylim(mp1_min, mp1_max)
        mp2_ax.set_ylim(mp2_min, mp2_max)
        
        mp1_ax.set(xlabel='Time (s)', ylabel='$B_T$ [T]')
        mp2_ax.set(xlabel='Time (s)', ylabel='$P_{OHM}$ [MW]')
        
        prof_ax.set(ylabel='$n_e$ [m$^{-3}$]', xlabel='R [m]')
        
        fig.suptitle('Subset of Pulse 83357')
        filename = f'{t}.png'
        filenames.append(filename)
        filenames.append(filename)
        filenames.append(filename)
        plt.savefig(filename)
        plt.close()
        
    with imageio.get_writer('windowed_gif.gif', mode='I') as writer:
        for filename in filenames: 
            image = imageio.imread(filename)
            writer.append_data(image)
    
    for filename in set(filenames):
        os.remove(filename)
        
    ff = ffmpy.FFmpeg(inputs={'windowed_gif.gif': None}, outputs={'windowed_mp4.mp4': None})
    ff.run()
    


In [238]:
t1, t2 = 54.592999, 56.497898
label_dict = {'BT': '$B_T$ [T]', 'CR0': 'a [m]', 'ELER': '$\Gamma \; (10^{22}$ e/s)', 'ELON': '$\kappa$ [-]', 'POHM': '$P_{OHM}$ [MW]', 'P_ICRH': '$P_{ICRH}$ [MW]', 'P_NBI': '$P_{NBI}$ [MW]', 'Q95' :'$q_{95}$ [-]', 'RGEO': '$R_{geo}$ [m]', 'TRIL': '$\delta_L$', 'TRIU': '$\delta_U$', 'VOLM': '$V_P$ [m$^{-3}$]', 'XIP': '$I_P$ [MA]'}
with h5py.File('/home/kitadam/ENR_Sven/moxie/data/processed/profile_database_v1_psi22.hdf5', 'r') as file:
    # print(file.keys())
    pulse_sample = file['83357']
    # print(pulse_sample.keys())
    sample_mp = pulse_sample['machine_parameters']
    sample_prof = pulse_sample['profiles']
    print(list(sample_mp.keys()))
    prof_times = sample_prof['time'][:]
    profiles = sample_prof['NE'][:]
    window_times, windowed_profiles = get_window_profile_times(profiles, prof_times, t1, t2)
    # print(window_times[2] - window_times[1])
    
    
    
    bt_val, bt_time = sample_mp['BT']['values'][:], sample_mp['BT']['time'][:]
    # new_bt_val, new_bt_time = get_window_mparams(bt_val, bt_time, t1, t2)
    final_bt_vals = average_machine_with_times(window_times, bt_val, bt_time)
    
    
    pohm_val, pohm_time = sample_mp['POHM']['values'][:]*1e-6, sample_mp['POHM']['time'][:]
    # new_pohm_val, new_bt_time = get_window_mparams(bt_val, bt_time, t1, t2)
    final_pohm_vals = average_machine_with_times(window_times, pohm_val, pohm_time)
    
    create_prof_mp_gif(windowed_profiles, sample_prof['radius'][:], window_times, bt_val, bt_time, pohm_val, pohm_time, final_bt_vals, final_pohm_vals, t1, t2)
    
    for key in sample_mp:
        break
        mp_val, mp_time = sample_mp[key]['values'][:], sample_mp[key]['time'][:]

        print(key)
        
        if key == 'XIP': 
            mp_val *= -1
            mp_val *= 1e-6
            # new_val *= -1
            
        if key == 'P_NBI' or key == 'P_ICRH' or key == 'POHM': 
            mp_val *= 1e-6
        
        if key == 'ELER': 
            mp_val *= 1e-22
        new_val, new_time = get_window_mparams(mp_val, mp_time, t1, t2)
        
        
        fig, ax = plt.subplots(figsize=[5, 4], dpi=400)
        ax.scatter(mp_time, mp_val)
        
        if key == 'POHM': 
            axins = ax.inset_axes([0.55, 0.45, 0.47, 0.47])
        else: 
            axins = ax.inset_axes([0.15, 0.1, 0.47, 0.47])
        axins.hlines(np.mean(new_val), t1, t2, color='green', label='Avg: {:.4}'.format(np.mean(new_val)))

        axins.fill_between(np.linspace(t1, t2), y1 = np.mean(new_val) - np.std(new_val), y2 = np.mean(new_val) + np.std(new_val), color='grey', alpha=0.3)
        axins.scatter(new_time, new_val)
        axins.set_xlim(t1 - 0.2, t2 +0.2)
        axins.set_ylim(min(new_val) - 3*np.std(new_val), max(new_val) + 3*np.std(new_val))

        ax.indicate_inset_zoom(axins, edgecolor='red')
        axins.set_xticklabels([])
        axins.set_yticklabels([])
        axins.legend()
        plt.ylabel(label_dict[key])
        plt.xlabel('Time (s)')
        
        ax.vlines([t1, t2], 0, max(mp_val), color='black', linestyle='dashed')
        # plt.title(key)
        
        
        
        fig, axs = plt.subplots(1, 2, figsize=[8, 4], sharey=True, constrained_layout=True,  dpi=400)
        
        axs[0].scatter(new_time, new_val)
        y_min, y_max = min(new_val) - 2*np.std(new_val), max(new_val) + 2*np.std(new_val)
        axs[0].set_xlim(t1 - 0.2, t2 +0.2)
        axs[0].set_ylim(y_min, y_max)
        axs[0].vlines(window_times, y_min, y_min + np.std(new_val), color='green', alpha=0.3, label='HRTS Frequency')
        axs[0].set_title('Before Windowing \n Avg: {:.4} $\pm$ {:.4}'.format(np.mean(new_val), np.std(new_val)))
        
        final_mp_vals = average_machine_with_times(window_times, mp_val, mp_time)
        axs[0].legend()
        axs[1].scatter(window_times, final_mp_vals)
        axs[1].set_ylim(y_min, y_max)
        axs[1].set_xlim(t1 - 0.2, t2 +0.2)
        axs[1].set_title('After Windowing \n Avg: {:.4} $\pm$ {:.4}'.format(np.mean(final_mp_vals), np.std(final_mp_vals)))
        fig.supxlabel('Time (s)')
        fig.supylabel(label_dict[key])
        # fig.text(0.04, 0.5, label_dict[key], va='center', rotation='vertical')
        plt.show()
        
        
        break

['BT', 'CR0', 'ELER', 'ELON', 'POHM', 'P_ICRH', 'P_NBI', 'Q95', 'RGEO', 'TRIL', 'TRIU', 'VOLM', 'XIP']


In [ ]:
# Units for each 
# Normailiztion for ELER, P, XIP 
# pROFILE SLICES 